In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np

# Import matplotlib libraries
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

# Some modules to display an animation using imageio.
import imageio
from IPython.display import HTML

import cv2 
import json
import time
import math

In [3]:
# Dictionary that maps from joint names to keypoint indices.
KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

# Maps bones to a matplotlib color name.
KEYPOINT_EDGE_INDS_TO_COLOR = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

def _keypoints_and_edges_for_display(keypoints_with_scores,
                                     height,
                                     width,
                                     keypoint_threshold=0.11):
  """Returns high confidence keypoints and edges for visualization.

  Args:
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    height: height of the image in pixels.
    width: width of the image in pixels.
    keypoint_threshold: minimum confidence score for a keypoint to be
      visualized.

  Returns:
    A (keypoints_xy, edges_xy, edge_colors) containing:
      * the coordinates of all keypoints of all detected entities;
      * the coordinates of all skeleton edges of all detected entities;
      * the colors in which the edges should be plotted.
  """
  keypoints_all = []
  keypoint_edges_all = []
  edge_colors = []
  num_instances, _, _, _ = keypoints_with_scores.shape
  for idx in range(num_instances):
    kpts_x = keypoints_with_scores[0, idx, :, 1]
    kpts_y = keypoints_with_scores[0, idx, :, 0]
    kpts_scores = keypoints_with_scores[0, idx, :, 2]
    kpts_absolute_xy = np.stack(
        [width * np.array(kpts_x), height * np.array(kpts_y)], axis=-1)
    kpts_above_thresh_absolute = kpts_absolute_xy[
        kpts_scores > keypoint_threshold, :]
    keypoints_all.append(kpts_above_thresh_absolute)

    for edge_pair, color in KEYPOINT_EDGE_INDS_TO_COLOR.items():
      if (kpts_scores[edge_pair[0]] > keypoint_threshold and
          kpts_scores[edge_pair[1]] > keypoint_threshold):
        x_start = kpts_absolute_xy[edge_pair[0], 0]
        y_start = kpts_absolute_xy[edge_pair[0], 1]
        x_end = kpts_absolute_xy[edge_pair[1], 0]
        y_end = kpts_absolute_xy[edge_pair[1], 1]
        line_seg = np.array([[x_start, y_start], [x_end, y_end]])
        keypoint_edges_all.append(line_seg)
        edge_colors.append(color)
  if keypoints_all:
    keypoints_xy = np.concatenate(keypoints_all, axis=0)
  else:
    keypoints_xy = np.zeros((0, 17, 2))

  if keypoint_edges_all:
    edges_xy = np.stack(keypoint_edges_all, axis=0)
  else:
    edges_xy = np.zeros((0, 2, 2))
  return keypoints_xy, edges_xy, edge_colors


def draw_prediction_on_image(
    image, keypoints_with_scores, crop_region=None, close_figure=False,
    output_image_height=None):
  """Draws the keypoint predictions on image.

  Args:
    image: A numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    crop_region: A dictionary that defines the coordinates of the bounding box
      of the crop region in normalized coordinates (see the init_crop_region
      function below for more detail). If provided, this function will also
      draw the bounding box on the image.
    output_image_height: An integer indicating the height of the output image.
      Note that the image aspect ratio will be the same as the input image.

  Returns:
    A numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  # To remove the huge white borders
  fig.tight_layout(pad=0)
  ax.margins(0)
  ax.set_yticklabels([])
  ax.set_xticklabels([])
  plt.axis('off')

  im = ax.imshow(image)
  line_segments = LineCollection([], linewidths=(4), linestyle='solid')
  ax.add_collection(line_segments)
  # Turn off tick labels
  scat = ax.scatter([], [], s=60, color='#FF1493', zorder=3)

  (keypoint_locs, keypoint_edges,
   edge_colors) = _keypoints_and_edges_for_display(
       keypoints_with_scores, height, width)

  line_segments.set_segments(keypoint_edges)
  line_segments.set_color(edge_colors)
  if keypoint_edges.shape[0]:
    line_segments.set_segments(keypoint_edges)
    line_segments.set_color(edge_colors)
  if keypoint_locs.shape[0]:
    scat.set_offsets(keypoint_locs)

  if crop_region is not None:
    xmin = max(crop_region['x_min'] * width, 0.0)
    ymin = max(crop_region['y_min'] * height, 0.0)
    rec_width = min(crop_region['x_max'], 0.99) * width - xmin
    rec_height = min(crop_region['y_max'], 0.99) * height - ymin
    rect = patches.Rectangle(
        (xmin,ymin),rec_width,rec_height,
        linewidth=1,edgecolor='b',facecolor='none')
    ax.add_patch(rect)

  fig.canvas.draw()
  image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
  image_from_plot = image_from_plot.reshape(
      fig.canvas.get_width_height()[::-1] + (3,))
  plt.close(fig)
  if output_image_height is not None:
    output_image_width = int(output_image_height / height * width)
    image_from_plot = cv2.resize(
        image_from_plot, dsize=(output_image_width, output_image_height),
         interpolation=cv2.INTER_CUBIC)
  return image_from_plot

def to_gif(images, duration):
  """Converts image sequence (4D numpy array) to gif."""
  imageio.mimsave('./animation.gif', images, duration=duration)
  return embed.embed_file('./animation.gif')

def progress(value, max=100):
  return HTML("""
      <progress
          value='{value}'
          max='{max}',
          style='width: 100%'
      >
          {value}
      </progress>
  """.format(value=value, max=max))

In [4]:
model_name = "movenet_thunder"

if "movenet_lightning" in model_name:
  module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/2")
  input_size = 192
elif "movenet_thunder" in model_name:
  module = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/2")
  input_size = 256
else:
  raise ValueError("Unsupported model name: %s" % model_name)

def movenet(input_image):
  """Runs detection on an input image.

  Args:
    input_image: A [1, height, width, 3] tensor represents the input image
      pixels. Note that the height/width should already be resized and match the
      expected input resolution of the model before passing into this function.

  Returns:
    A [1, 1, 17, 3] float numpy array representing the predicted keypoint
    coordinates and scores.
  """
  model = module.signatures['serving_default']

  # SavedModel format expects tensor type of int32.
  input_image = tf.cast(input_image, dtype=tf.int32)
  # Run model inference.
  outputs = model(input_image)
  # Output is a [1, 1, 17, 3] tensor.
  keypoints_with_scores = outputs['output_0'].numpy()
  return keypoints_with_scores

2023-11-27 18:54:39.463686: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-11-27 18:54:39.463708: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-11-27 18:54:39.463712: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-11-27 18:54:39.463746: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-27 18:54:39.463767: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:

from math import atan
# Function to find the
# angle between two lines
def findAngle(point1, point2, point3):
    M1 = (point1[1]-point2[1])/(point1[0]-point2[0])
    M2 = (point3[1]-point2[1])/(point3[0]-point2[0])
    PI = 3.14159265
	
	# Store the tan value of the angle
    angle = abs((M2 - M1) / (1 + M1 * M2))

	# Calculate tan inverse of the angle
    ret = atan(angle)

	# Print the result
    return (round(ret, 2))

In [7]:

def calculate_arm_angle(hip, shoulder, wrist):
    theta1 = math.atan2(hip[1] - shoulder[1], hip[0] - shoulder[0])
    theta2 = math.atan2(wrist[1] - shoulder[1], wrist[0] - shoulder[0])
    
    arm_angle = theta2 - theta1
    return arm_angle

In [8]:
vid = cv2.VideoCapture(0) 
  
while(True):
    ret, frame = vid.read() 

    image_data = frame
    
    image = tf.convert_to_tensor(image_data, dtype=tf.uint8)
    input_image = tf.expand_dims(image, axis=0)
    input_image = tf.image.resize_with_pad(input_image, input_size, input_size)

    keypoints_with_scores = movenet(input_image)

    Lshoulder = keypoints_with_scores[0][0][5]
    Lelbow = keypoints_with_scores[0][0][7]
    Lwrist = keypoints_with_scores[0][0][9]
    Lhip = keypoints_with_scores[0][0][11]
    LAnkle = keypoints_with_scores[0][0][15]
    LeftEar = keypoints_with_scores[0][0][3]
    LKnee = keypoints_with_scores[0][0][13]
    LeftEye = keypoints_with_scores[0][0][1]

    RightEar = keypoints_with_scores[0][0][4]
    Rshoulder = keypoints_with_scores[0][0][6]
    Relbow = keypoints_with_scores[0][0][10]
    Rwrist = keypoints_with_scores[0][0][9]
    Rhip = keypoints_with_scores[0][0][12]
    RAnkle = keypoints_with_scores[0][0][16]
    RKnee = keypoints_with_scores[0][0][14]
    RightEye = keypoints_with_scores[0][0][2]
    
    Lshoulder_angle = calculate_arm_angle(Lelbow, Lshoulder, Lhip)
    LElbow_angle= findAngle(Lshoulder , Lelbow , Lwrist)
    #Rshoulder_angle = calculate_arm_angle(Relbow, Rshoulder, Rhip)
    RElbow_angle = findAngle(Rshoulder , Relbow , Rwrist)
    LKneePitch = findAngle(Lhip, LKnee, LAnkle)
    RKneePitch = LKneePitch
    Lhip = findAngle(Lshoulder, Lhip, LKnee)
    Rhip = Lhip

    if  abs(Lshoulder_angle) > 0.7 :
            data = { 
                #"RShoulderPitch": (Rshoulder_angle-1.57) , 
                "LElbowRoll": LElbow_angle , 
                "RElbowRoll": RElbow_angle, 
                "LShoulderPitch": -(Lshoulder_angle-1.57),
                "LKneePitch" : LKneePitch,
                "RKneePitch" : RKneePitch,
                "LHipPitch" : -Lhip,
                "RHipPitch" : -Rhip,
                }
    else:
            data = { 
                "LShoulderPitch": -(Lshoulder_angle-1.57),
                "LKneePitch" : LKneePitch,
                "RKneePitch" : RKneePitch,
                "LHipPitch" : -Lhip,
                "RHipPitch" : -Rhip,
                }
    with open('angles_side.json', 'w') as json_file:   
        json.dump(data, json_file)
    time.sleep(1)

    display_image = tf.expand_dims(image, axis=0)
    display_image = tf.cast(tf.image.resize_with_pad(
        display_image, 1280, 1280), dtype=tf.int32)
    output_overlay = draw_prediction_on_image(
        np.squeeze(display_image.numpy(), axis=0), keypoints_with_scores)
    cv2.imshow('output_overlay' ,output_overlay)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
vid.release() 
cv2.destroyAllWindows()

2023-11-27 18:55:14.049823: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/var/folders/6m/4pp7tnxj4s318p8vwlk76fgw0000gp/T/ipykernel_16775/96334677.py:160: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed two minor releases later. Use buffer_rgba instead.
  image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
/var/folders/6m/4pp7tnxj4s318p8vwlk76fgw0000gp/T/ipykernel_16775/96334677.py:160: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed two minor releases later. Use buffer_rgba instead.
  image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
/var/folders/6m/4pp7tnxj4s318p8vwlk76fgw0000gp/T/ipykernel_16775/96334677.py:160: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed two minor releas

KeyboardInterrupt: 